In [2]:
df=pd.read_csv("stable-diffusion\metadata.csv")

In [3]:
df

,image_path,description
0,bracelet/$_3.jpeg,A bronze bracelet featuring a golden sphere be...
1,bracelet/00G0FPIT4D9J.jpg,A stack of eight bracelets in varying shades o...
2,bracelet/01D7PTXV8C7F.jpg,"A pair of matte black metal cuff bracelets, on..."
3,bracelet/02G14031IMEF.jpg,A gold-toned bangle bracelet encrusted with sm...
4,bracelet/031Q9F07X716.jpg,A gold bracelet with a lion's head centerpiece...
...,...,...
6151,earring_best/028_001.png,Close-up shot of a pair of gold lightning bolt...
6152,earring_best/031_020.png,A pair of gold earrings with textured detailin...
6153,earring_best/031_033.png,Pair of ornate gold earrings with teardrop-sha...
6154,earring_best/031_034.png,Pair of ornate gold earrings with intricate de...


In [4]:
import pandas as pd
import json
import os

# Define paths
dataset_dir = "stable-diffusion\jewellery_dataset"  # Adjust this to your dataset directory
csv_path = "stable-diffusion\metadata.csv"   # Adjust this to your CSV file location

# Ensure the dataset directory exists
os.makedirs(dataset_dir, exist_ok=True)

# Load the CSV
df = pd.read_csv(csv_path)

# Create metadata.jsonl
metadata_path = os.path.join(dataset_dir, "metadata.jsonl")
with open(metadata_path, "w") as f:
    for _, row in df.iterrows():
        file_name = row["image_path"]  # e.g., "bracelets/bracelet1.jpg"
        caption = row["description"]
        json_line = {"file_name": file_name, "text": caption}
        f.write(json.dumps(json_line) + "\n")

In [8]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Prints GPU name

True
NVIDIA RTX A6000


In [6]:
!git clone https://github.com/huggingface/diffusers.git

Cloning into 'diffusers'...
Updating files:  46% (912/1982)
Updating files:  47% (932/1982)
Updating files:  48% (952/1982)
Updating files:  49% (972/1982)
Updating files:  50% (991/1982)
Updating files:  51% (1011/1982)
Updating files:  52% (1031/1982)
Updating files:  53% (1051/1982)
Updating files:  54% (1071/1982)
Updating files:  55% (1091/1982)
Updating files:  56% (1110/1982)
Updating files:  57% (1130/1982)
Updating files:  58% (1150/1982)
Updating files:  59% (1170/1982)
Updating files:  60% (1190/1982)
Updating files:  61% (1210/1982)
Updating files:  62% (1229/1982)
Updating files:  63% (1249/1982)
Updating files:  64% (1269/1982)
Updating files:  65% (1289/1982)
Updating files:  66% (1309/1982)
Updating files:  67% (1328/1982)
Updating files:  68% (1348/1982)
Updating files:  69% (1368/1982)
Updating files:  70% (1388/1982)
Updating files:  71% (1408/1982)
Updating files:  72% (1428/1982)
Updating files:  73% (1447/1982)
Updating files:  74% (1467/1982)
Updating files:  75%

In [18]:
!accelerate launch \
  --num_processes=1 \
  --num_machines=1 \
  --mixed_precision=fp16 \
  --dynamo_backend=no \
  diffusers/examples/text_to_image/train_text_to_image_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="stable-diffusion/jewellery_dataset" \
  --output_dir="output" \
  --mixed_precision="fp16" \
  --resolution=1024 \
  --train_batch_size=1 \
  --learning_rate=1e-5 \
  --max_train_steps=10000 \
  --checkpointing_steps=1000 \
  --validation_prompt="a beautiful necklace" \
  --validation_epochs=1

c:\Users\rohan-graph-rkk\Desktop\Stable_Diffusion\stable_diff\lib\site-packages\accelerate\accelerator.py:506: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
04/26/2025 14:46:50 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'dynamic_thresholding_ratio', 'thresholding', 'variance_type', 'clip_sample_range', 'rescale_betas_zero_snr'} was not found in config. Values will be initialized to default values.
{'shift_factor', 'latents_

In [23]:
import diffusers
print([name for name in dir(diffusers) if "Pipeline" in name])

[]


In [22]:
import numpy as np
from diffusers import StableDiffusionXLPipeline, AutoencoderKL
from diffusers.utils import load_image
from PIL import Image
import torch

# Load the pipeline with the SDXL base model
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
)

# Load fine-tuned LoRA weights from the latest checkpoint
pipe.load_lora_weights("output/checkpoint-9000")

# Optional: Use a better VAE for improved quality
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)
pipe.vae = vae

# Optimize for low GPU memory
pipe.enable_model_cpu_offload()

# Define your test prompts
prompt = "a diamond necklace with intricate details"
negative_prompt = "blurry, low quality"

# Generate the image (without ControlNet for now)
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=50,
    guidance_scale=7.5,
).images[0]

# Save and display the generated image
image.save("test_generated_image.png")
from IPython.display import display
display(image)

ImportError: cannot import name 'StableDiffusionXLPipeline' from 'diffusers' (unknown location)

In [24]:
from diffusers.pipelines import stable_diffusion
print([name for name in dir(stable_diffusion) if "Pipeline" in name])

ModuleNotFoundError: No module named 'diffusers.pipelines'